## Deap


Библиотека Neurokit2

In [9]:
import mne
import os
from tqdm import tqdm_notebook
import neurokit2 as nk
import numpy as np
import pickle
import yasa

Порядок каналов из архива

http://www.eecs.qmul.ac.uk/mmv/datasets/deap/readme.html

data	40 x 40 x 8064	video/trial x channel x data

labels	40 x 4	video/trial x label (valence, arousal, dominance, liking)

Описание функции nk.complexity()
https://neurokit2.readthedocs.io/en/latest/functions.html#module-neurokit2.complexity

In [ ]:
n_subjects = 32
n_films = 40
n_channels = 18
ch_order = [18,    2,    3,   6,    7,    15,   10,  11,   13,   14,   31,  28,   29,   23,  24,   25,   19,    20]
ch_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(1 , n_subjects + 1)):
    Subject = ('0' + str(Subject)) if Subject < 10 else str(Subject)
    raw = pickle.load(open('G:/Мой диск/Research/DEAP/s{}.dat'.foramt(Subject), 'rb'), encoding='latin1')
    for film in tqdm_notebook(range(n_films)):
        for channel in tqdm_notebook(range(n_channels)):
            if os.path.isfile('G:/Мой диск/Research/features_deap_neurokit/S_{}_F_{}_{}_.xlsx'.format(Subject, film, ch_names[channel])):
                continue
            # Просмотр начинается с 384 сэмпла
            df, info = nk.complexity(raw['data'][film][ch_order[channel]][384:], which = ['fast', 'medium'])
            df['Subj'] =  Subj
            df['film'] = film
            df['ch'] = ch_names[channel]
            df['labels'] = raw['labels'][film][3]
            df.to_excel('G:/Мой диск/Research/Project2022/features_deap_40_128_neurokit/S_{}_F_{}_{}_.xlsx'.format(Subject, film, ch_names[channel]), index=False)

## Испытуемые из лаборатории

Neurokit2

Рейтинги : по стобцам фильмы, по строчкам испытуемые. У 11 исп. отсутсвуют два рейтинга.

In [6]:
ratings = np.array(
       [
       [10., 10.,  2.,  5.,  9.,  9.,  5.,  9.],
       [10.,  8.,  7.,  9.,  7.,  6.,  6.,  9.],
       [ 6.,  3.,  5.,  6.,  5.,  5.,  5.,  2.],
       [ 8.,  6.,  5.,  9.,  5.,  7.,  4.,  6.],
       [ 3.,  2.,  4.,  1.,  7.,  3.,  7.,  2.],
       [ 7.,  7.,  2.,  8.,  5.,  3.,  2.,  5.],
       [ 9.,  4.,  4.,  3.,  9.,  8.,  8.,  5.],
       [ 5.,  7.,  5.,  9.,  3.,  2.,  8.,  7.],
       [ 8.,  8.,  2.,  3.,  7.,  5.,  5.,  9.],
       [ 3.,  8.,  7.,  3.,  1.,  3.,  6.,  2.],
       [ 8., np.nan,  3.,  5.,  8., np.nan,  6.,  4.],
       [10.,  5.,  6.,  9., 10.,  7.,  8.,  2.]
       ]
)

In [ ]:
n_subjects = 12
n_films = 8
n_channels = 18
channel_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    for film in tqdm_notebook(range(1, n_films + 1)):
        if not os.path.isfile('G:/Мой диск/Research/Project2022/Cuts_filtered_40_128/{}/filtered_watch_{}.fif'.format(Subject, film)):
            continue

        # читаем просмотр из файла
        
        raw = mne.io.read_raw('G:/Мой диск/Research/Project2022/Cuts_filtered_40_128/{}/filtered_watch_{}.fif'.format(Subject, film))
        
        #удаляем ненужные каналы
        
        if 'VEOG' in raw.ch_names:
            raw.drop_channels(['VEOG'])
        if 'STI' in raw.ch_names:
            raw.drop_channels(['STI'])
        raw.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])
        
        for channel in tqdm_notebook(channel_names):
            if os.path.isfile('G:/Мой диск/Research/Project2022/features_40_128_neurokit/S_{}_F_{}_{}_.xlsx'.format(Subject, film, channel)):
                continue
            df, info = nk.complexity(raw[channel][0][0], which = ['fast', 'medium'])
            df['Subj'] =  Subject
            df['film'] = film
            df['ch'] = channel
            df['labels'] = ratings[i][j]
            df.to_excel('G:/Мой диск/Research/Project2022/features_40_128_neurokit/S_{}_F_{}_{}_.xlsx'.format(Subject, film, channel), index=False)

Мощности частот (библиотека Yasa) https://raphaelvallat.com/yasa/build/html/generated/yasa.bandpower_from_psd.html?highlight=bandpower#yasa.bandpower_from_psd

Мы добавляем построчно каждый из каналов

In [ ]:
n_subjects = 12
n_films = 8
n_channels = 18
channel_names = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']
for Subject in tqdm_notebook(range(n_subjects)):
    frames = []
    frames_rel = []
    for film in tqdm_notebook(range(n_films)):
        if not os.path.isfile('G:/Мой диск/Research/Project2022/Cuts_filtered_40_128/{}/filtered_relax_before_{}.fif'.format(Subject, film)):
            continue
        relax = mne.io.read_raw('G:/Мой диск/Research/Project2022/Cuts_filtered_40_128/{}/filtered_relax_before_{}.fif'.format(Subject, film))
        watch = mne.io.read_raw('G:/Мой диск/Research/Project2022/Cuts_filtered_40_128/{}/filtered_watch_{}.fif'.format(Subject, film))
        
        sf = relax.info['sfreq']
        win = int(4 * sf)
        
        if 'VEOG' in relax.ch_names:
            relax.drop_channels(['VEOG'])
            watch.drop_channels(['VEOG'])
        if 'STI' in relax.ch_names:
            relax.drop_channels(['STI'])
            watch.drop_channels(['STI'])
        relax.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])
        watch.drop_channels(['Pulse', 'Zygom', 'Corr', 'EDA', 'Mark'])
        

        freqs_relax, psd_relax = welch(relax.get_data(), sf, nperseg=win, average='median')
        freqs_watch, psd_watch = welch(watch.get_data(), sf, nperseg=win, average='median')
        
        bp_relax = yasa.bandpower_from_psd(psd_relax, freqs_relax, channel_names, bands=
                                   [(4, 8, 'Theta'), (8, 12, 'Alpha'), (12, 30, 'Beta')], relative=False).drop(columns=['FreqRes', 'Relative', 'TotalAbsPow'])
        bp_watch = yasa.bandpower_from_psd(psd_watch, freqs_watch, channel_names, bands=
                                   [(4, 8, 'Theta'), (8, 12, 'Alpha'), (12, 30, 'Beta')], relative=False).drop(columns=['FreqRes', 'Relative', 'TotalAbsPow'])
        bp_ratio_relax = yasa.bandpower(relax, relax.info['sfreq'], bands=
                                   [(4, 8, 'Theta_ratio'), (8, 12, 'Alpha_ratio'), (12, 30, 'Beta_ratio')], relative=True).drop(columns=['FreqRes', 'Relative'])
        bp_ratio_watch = yasa.bandpower(watch, watch.info['sfreq'], bands=
                                   [(4, 8, 'Theta_ratio'), (8, 12, 'Alpha_ratio'), (12, 30, 'Beta_ratio')], relative=True).drop(columns=['FreqRes', 'Relative'])

        for channel in tqdm_notebook(range(n_channels)):
            df_relax = pd.DataFrame(bp_relax.loc[channel]).transpose()
            df_watch = pd.DataFrame(bp_watch.loc[channel]).transpose()
            dr = pd.DataFrame(p_relax.loc[relax.ch_names[channel]]).transpose()
            dr.rename(index={relax.ch_names[channel]:channel}, inplace=True)
            df_relax = df_relax.join(dr)
            dw = pd.DataFrame(p_watch.loc[watch.ch_names[channel]]).transpose()
            dw.rename(index={relax.ch_names[channel]:channel}, inplace=True)
            df_watch = df_watch.join(dw)
            df_relax.rename(index={channel:'F' + film + '_relax'}, inplace=True)
            df_watch.rename(index={channel:'F' + film + '_watch'}, inplace=True)

            df_rel = {
                'Theta_watch-relax' : float(df_watch['Theta']) - float(df_relax['Theta']),
                'Alpha_watch-relax' : float(df_watch['Alpha']) - float(df_relax['Alpha']),
                'Beta_watch-relax' : float(df_watch['Beta']) - float(df_relax['Beta']),
            }
            df_rel = pd.DataFrame(df_rel, index=['F' + film + '_' + watch.ch_names[j]])
            df_div = {
                'Beta/Alpha' : float(df_rel['Beta_watch-relax']) / float(df_rel['Alpha_watch-relax']),
                'Beta/(Alpha + Theta)' : float(df_rel['Beta_watch-relax']) / (float(df_rel['Alpha_watch-relax'] + float(df_rel['Theta_watch-relax']))),
            }
            df_div = pd.DataFrame(df_div, index=['F' + film + '_' + watch.ch_names[channel]])
            df_all = df_rel.join(df_div)
            frames_rel.append(df_all)
            frames.append(df_relax)
            frames.append(df_watch)

    result_rel = pd.concat(frames_rel)
    result_rel.to_excel('G:/Мой диск/Research/Project2022/features_yasa/S' + Subject + '_ratio.xlsx')
    result = pd.concat(frames)
    result.to_excel('G:/Мой диск/Research/Project2022/features_yasa/S' + Subject + '.xlsx')